In [13]:
# Implementing pyradox from github repo of ritvik19. To run the file, run this command in the terminal first:
# pip install git+https://github.com/Ritvik19/pyradox.git
# the pyradox library contains diffrent CNNs created. Here, we used MobileNetV3. 
# TODO: Implement using ResNet, VGG and others.

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf 
from keras.models import Sequential, Model
from keras.layers import Activation, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
from tensorflow.keras import layers, callbacks, utils, applications, optimizers, Input
from tensorflow.keras.models import Sequential, Model, load_model
from pyradox import convnets

ModuleNotFoundError: No module named 'pyradox'

In [2]:
train_file = '/Users/harshjhunjhunwala/Desktop/github/facial_keypoint/data/training.csv'
test_file = '/Users/harshjhunjhunwala/Desktop/github/facial_keypoint/data/test.csv'
lookup_file = '/Users/harshjhunjhunwala/Desktop/github/facial_keypoint/data/IdLookupTable.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
lookup = pd.read_csv(lookup_file)

In [3]:
def cleaning(train):
    train.isnull().any().value_counts()
    train.fillna(method = 'ffill',inplace = True)
    image_list = []
    for i in range(len(train)):
        img = train['Image'][i].split(' ')
        img = ['0' if x == '' else x for x in img]
        image_list.append(img)
    image_list = np.array(image_list,dtype = 'float')
    return image_list.reshape(-1,96,96,1)    

In [4]:
X_train = cleaning(train)
X_test = cleaning(test)

In [5]:
plt.imshow(image_list[0].reshape(96,96 ),cmap='gray')
plt.show()

NameError: name 'image_list' is not defined

In [ ]:
keypoint_data = train.drop('Image', axis=1)
fac_points = []
for i in range(len(keypoint_data)):
    fac_points.append(keypoint_data.iloc[i])
y_train = fac_points

fac_points = np.array(fac_points, dtype='float')
y_train = np.array(y_train, dtype="float")


In [ ]:
###Visualising facial keypoints

# x and y coords are alternating so I'll try to add a scatter plot with the x&y lists as alternating values 
len(fac_points[0])

In [ ]:
plt.imshow(image_list[0].reshape(96,96 ),cmap='gray')
plt.scatter(fac_points[0][0::2], fac_points[0][1::2])
plt.show()

In [29]:
# Training the Model

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 1)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(30))
model.summary()

# model2 stats -> acc 69; loss 69

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 94, 94, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)            

In [12]:
inputs = Input(shape=(96, 96, 1))
x = layers.Convolution2D(3, (1, 1), padding='same')(inputs)
x = layers.LeakyReLU(alpha = 0.1)(x)
x = convnets.MobileNetV3(config='small')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(30)(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

NameError: name 'convnets' is not defined

In [30]:

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

In [31]:
model.fit(X_train,y_train,epochs = 5,batch_size = 32,validation_split = 0.2)

Train on 5639 samples, validate on 1410 samples
Epoch 1/5
5639/5639 [==============================] - 93s 16ms/step - loss: 409.7329 - acc: 0.5295 - val_loss: 109.2181 - val_acc: 0.6844
Epoch 2/5
5639/5639 [==============================] - 72s 13ms/step - loss: 93.9890 - acc: 0.6774 - val_loss: 97.1307 - val_acc: 0.6702
Epoch 3/5
5639/5639 [==============================] - 68s 12ms/step - loss: 87.7357 - acc: 0.6436 - val_loss: 76.1869 - val_acc: 0.6326
Epoch 4/5
5639/5639 [==============================] - 77s 14ms/step - loss: 75.5060 - acc: 0.6927 - val_loss: 75.3688 - val_acc: 0.6362
Epoch 5/5
5639/5639 [==============================] - 85s 15ms/step - loss: 69.9901 - acc: 0.6916 - val_loss: 94.6305 - val_acc: 0.6851


In [32]:
predict = model.predict(X_test)

NameError: name 'X_test' is not defined

In [ ]:
FeatureName = list(IdLookupTable['FeatureName'])
ImageId = list(IdLookupTable['ImageId']-1)
RowId = list(IdLookupTable['RowId'])


# predict results
predict = list(predict)


Data = []
for i in list(FeatureName):
    Data.append(FeatureName.index(i))
    
    
Data_Pre = []
for x,y in zip(ImageId,Data):
    Data_Pre.append(predict[x][y])
    
    





RowId = pd.Series(RowId,name = 'RowId')
Location = pd.Series(Data_Pre,name = 'Location')

submission = pd.concat([RowId,Location],axis = 1)
submission.to_csv('./Submit.csv',index=False)
